In [3]:
# !pip install sqlalchemy
# !pip install sqlacodegen

In [4]:
import pickle
import pandas as pd

In [5]:
data = [{'name': 'Ajay'}]

with open('users.pk1', 'wb') as f:
    pickle.dump(data, f)

In [6]:
read_data = pickle.load(open('users.pk1', 'rb'))
print(read_data, type(read_data))

[{'name': 'Ajay'}] <class 'list'>


In [7]:
df = pd.DataFrame(data)
df.head()
df.dtypes

name    object
dtype: object

In [8]:
df.to_pickle('df_pkl')

In [9]:
df2 = pd.read_pickle('df_pkl')
df2.head()

,name
0,Ajay


In [10]:
df2.dtypes

name    object
dtype: object

In [11]:
df.to_csv('my_csv', index=False)

In [12]:
df3 = pd.read_csv('my_csv')
df3.head()

,name
0,Ajay


In [13]:
df3.dtypes

name    object
dtype: object

# SQLAlchemy

In [14]:
from dataclasses import dataclass

@dataclass
class Movie:
    name:str = 'Unknown'
    genre:str = 'Action'
    
#     def __init__(self, name=''):
#         self.name = name

In [15]:
my_movie = Movie(name='SkR')
my_movie.name

'SkR'

In [16]:
my_movie.genre

'Action'

In [17]:
data = [
    {
        'name': 'King Kong',
        'genre': 'Action'
    },
    {
        'name': 'Passenger',
        'genre': 'Romantic'
    },
    {
        'name': 'Avenger',
        'genre': 'Action'
    },
    {
        'name': 'Arrival',
        'genre': 'Sci-Fi'
    }
]


In [18]:
df = pd.DataFrame(data)
df.head()

,name,genre
0,King Kong,Action
1,Passenger,Romantic
2,Avenger,Action
3,Arrival,Sci-Fi


In [19]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String

In [20]:
engine = create_engine('sqlite:///app2.db')
Session = sessionmaker(bind=engine)
my_sess = Session()

In [21]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [22]:
class Movie(Base): # Table
    __tablename__ = 'movies'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    description = Column(String)
    year = Column(Integer, nullable=True)
    
    def __repr__(self):
        return f'<Movie name={self.name}>'

In [23]:
#  Add Table to Database
Base.metadata.create_all(engine)

In [24]:
movie_obj = Movie(name='Passenger', genre='Romance')
print(movie_obj.name)

Passenger


In [25]:
movie_obj.id

In [26]:
my_sess.add(movie_obj) # Prepare to Save
my_sess.commit() # Save

In [27]:
movie_obj.id

1

In [28]:
movie_obj2 = Movie(name='Avenger', genre='Action')
print(movie_obj2.name)

Avenger


In [29]:
movie_obj2.id

In [30]:
my_sess.add(movie_obj2)

In [31]:
movie_obj3 = Movie(name='King Kong', genre='Action')
print(movie_obj3.name)

King Kong


In [32]:
my_sess.add(movie_obj3)

In [33]:
my_sess.commit()

# Crud in sqlalchemy
Create
Retreive
Update
Delete

In [34]:
Session = sessionmaker(bind=engine)
session = Session()

In [35]:
#  Create

movie = Movie(name='Last')
session.add(movie)
session.commit()

In [36]:
# Retreive

# Get 1 Item
movie_a = session.query(Movie).get(2)
print(movie_a.id, movie_a.name, movie_a.description)

2 Avenger None


In [37]:
# List
qs = session.query(Movie).all()
print(qs)

[<Movie name=Passenger>, <Movie name=Avenger>, <Movie name=King Kong>, <Movie name=Last>]


In [38]:
# List & Filter by Column Value
qs = session.query(Movie).filter_by(name='Avenger').all()
qs

[<Movie name=Avenger>]

In [39]:
# List & Filter by Column Value Containing Somthing
qs = session.query(Movie).filter(Movie.name.contains('hello')).all()
qs

[]

In [40]:
# List & Filter by Column Value Containing Somthing
my_query = input('What are you looking for?\n') or 'Unknown'
qs = session.query(Movie).filter(Movie.name.contains(my_query)).all()
qs

What are you looking for?



[]

In [41]:
# Update

movie_a = session.query(Movie).get(2)
movie_a.description = 'I Love Iron man'
print(movie_a.id, movie_a.name, movie_a.description)
session.commit()

2 Avenger I Love Iron man


In [42]:
movie_a = session.query(Movie).get(2)
print(movie_a.id, movie_a.name, movie_a.description)

2 Avenger I Love Iron man


In [43]:
qs = session.query(Movie).filter(Movie.name.contains('Avenger')).all()
for movie_obj in qs:
    movie_obj.name = 'Avanger'

session.commit()

In [44]:
# Delete
movie_a = session.query(Movie).get(1)
session.delete(movie_a)
session.commit()
session.flush()

In [45]:
movie_a = session.query(Movie).get(3)
print(movie_a.id, movie_a.name)

3 King Kong


## Sqlacodegen And Pandas

In [46]:
!sqlacodegen sqlite:///app2.db

# coding: utf-8
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class Movie(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    description = Column(String)
    year = Column(Integer)


In [47]:
engine = create_engine('sqlite:///app2.db')
Session = sessionmaker(bind=engine)
session = Session()

In [55]:
# coding: utf-8
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class MovieApp2(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    description = Column(String)
    year = Column(Integer)
    
    def __repr__(self):
        return f'<Movie name={self.name}>'

In [56]:
qs = session.query(MovieApp2).all()

In [57]:
print(qs)

[<Movie name=Avanger>, <Movie name=King Kong>, <Movie name=Last>]
